<a href="https://colab.research.google.com/github/sparkgeo/workshops/blob/main/OGC_Madrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Hi! Welcome to the Sparkgeo OGC STAC Workshop. During this workshop you'll learn how to source, create, and manipulate a STAC item and collection. You'll also learn how to implement STAC extensions such as the scientific and label extensions. 

## What is STAC?

From the website: 
>The SpatioTemporal Asset Catalog (STAC) specification provides a common language to describe a range of geospatial information, so it can more easily be indexed and discovered. A 'spatiotemporal asset' is any file that represents information about the earth captured in a certain space and time.

STAC provides a 'signpost' for your dataset, giving those interested information about what the data is and where to find it. 

Lets get right into things!


# Canada Landsat Disturbance (CanLaD) 

In [51]:
!pip install rasterio
!pip install rio_cogeo
!pip install aiohttp
import rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 18.3 MB/s 
     |████████████████████████████████| 144 kB 39.6 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 271 kB 52.5 MB/s 


# Library Installation

We start by installing a couple python libraries that don't come pre-installed in the CoLab notebook. We will use PySTAC to create and manipulate our STAC data and Rasterio for some exploratory analysis.

### PySTAC
PySTAC is a library for working with STAC in Python 3. Some features of PySTAC:

>* Reading and writing STAC version 1.0.
* In-memory manipulation of STAC catalogs.
* Extend the I/O of STAC metadata to provide support for other platforms (e.g. cloud providers).
* Easy, efficient crawling of STAC catalogs. STAC objects are only read in when needed.

### PyProj
PyProj is a Python interface to PROJ (cartographic projections and coordinate transformations library). This is used for reprojecting coordinates later on in the tutorial.






In [29]:
!pip install pystac
!pip install pyproj
!pip install fsspec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Imports 

Next up we have our imports, some of these are for exploring the data, the others are for making sure our data is of the right type in the STAC object. We'll set an EPSG constant for later.


In [52]:
import pystac
from urllib.request import urlopen, urlretrieve
import requests
import aiohttp
import json
import fsspec
from typing import Any, Dict
from pyproj import CRS, Proj
from shapely.geometry import Polygon, box
from typing import Any, Dict, List, Optional
from datetime import datetime
from shapely.geometry import mapping as geojson_mapping
from pystac.extensions.file import FileExtension
from pystac import Link, Provider, ProviderRole

from pystac.extensions.projection import ProjectionExtension
from pystac.extensions.scientific import ScientificExtension
from pystac.extensions.label import (
    LabelClasses,
    LabelExtension,
    LabelTask,
    LabelType,
)

In [31]:
DATA_EPSG = 3979
DATA_CRS_WKT = CRS.from_epsg(DATA_EPSG).to_wkt()

## Cloud Optimized Geotiff

Lets grab some of the data from the Government of Canada website and make some COGS. COGS are great for cloud geospatial since they support range requests. Couple this with clever internal tiling and you have a very consumable tiff that is also backwards compatible.



In [32]:
!wget 'https://ftp.maps.canada.ca/pub/nrcan_rncan/Forests_Foret/landsat_disturbance/1984_2015/CanLaD_20151984_latest_YRT2.tif'
!wget 'https://ftp.maps.canada.ca/pub/nrcan_rncan/Forests_Foret/landsat_disturbance/1984_2015/CanLaD_20151984_latest_type.tif'
!ls

--2022-06-17 05:56:29--  https://ftp.maps.canada.ca/pub/nrcan_rncan/Forests_Foret/landsat_disturbance/1984_2015/CanLaD_20151984_latest_YRT2.tif
Resolving ftp.maps.canada.ca (ftp.maps.canada.ca)... 15.222.205.109
Connecting to ftp.maps.canada.ca (ftp.maps.canada.ca)|15.222.205.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 604432423 (576M) [image/tiff]
Saving to: ‘CanLaD_20151984_latest_YRT2.tif.2’

CanLaD_20151984_lat 100%[===================>] 576.43M  61.8MB/s    in 9.2s    

2022-06-17 05:56:39 (62.5 MB/s) - ‘CanLaD_20151984_latest_YRT2.tif.2’ saved [604432423/604432423]

--2022-06-17 05:56:39--  https://ftp.maps.canada.ca/pub/nrcan_rncan/Forests_Foret/landsat_disturbance/1984_2015/CanLaD_20151984_latest_type.tif
Resolving ftp.maps.canada.ca (ftp.maps.canada.ca)... 15.222.205.109
Connecting to ftp.maps.canada.ca (ftp.maps.canada.ca)|15.222.205.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290194588 (277M) [image/tiff]
Sa

In [33]:
!rio cogeo validate CanLaD_20151984_latest_type.tif

The following errors were found:
- The offset of the main IFD should be < 300. It is 241765672 instead
- The offset of the first block of overview of index 1 should be after the one of the overview of index 2
- The offset of the first block of overview of index 0 should be after the one of the overview of index 1
- The offset of the first block of the main resolution image should be after the one of the overview of index 2
/content/CanLaD_20151984_latest_type.tif is NOT a valid cloud optimized GeoTIFF


In [34]:
# THIS CAN TAKE A WHILE. USE CAUTION
#
# cog_translate('CanLaD_20151984_latest_type.tif', 'CanLaD_20151984_latest_type_cog.tif')
# !rio cogeo validate `CanLaD_20151984_latest_type_cog.tif`

In [35]:
# You can use this COG on AWS instead if needed.
!rio cogeo validate https://ogc-madrid.s3.eu-central-1.amazonaws.com/CanLaD_20151984_latest_type_cog.tiff

https://ogc-madrid.s3.eu-central-1.amazonaws.com/CanLaD_20151984_latest_type_cog.tiff is a valid cloud optimized GeoTIFF


I'll also declare this function which is a modified version of this script: https://github.com/stactools-packages/nrcan-landcover/blob/7708e8656834d23d910b7b8f65b5d7f520895224/src/stactools/nrcan_landcover/utils.py#L42
This script captures data from a provided `JSON-LD` metadata file, which we'll pass in later. The Canadian Federal datasets are great for providing this additional info and it makes our jobs much easier too. 

In [36]:
def get_metadata(metadata_url: str) -> Dict[str, Any]:
    """Gets metadata from the various formats published by NRCan.
    Args:
        metadata_url (str): url to get metadata from.
    Returns:
        dict: Metadata.
    """
    if metadata_url.endswith(".jsonld"):
        if metadata_url.startswith("http"):
            metadata_response = requests.get(metadata_url)
            jsonld_response = metadata_response.json()
        else:
            with open(metadata_url) as f:
                jsonld_response = json.load(f)

        tiff_metadata = [
            i for i in jsonld_response.get("@graph")
            if i.get("dct:format") == "TIFF"
        ][0]

        geom_obj = next(
            (x["locn:geometry"] for x in jsonld_response["@graph"]
             if "locn:geometry" in x.keys()),
            [],
        )  # type: Any
        geom_metadata = next(
            (json.loads(x["@value"])
             for x in geom_obj if x["@type"].startswith("http")),
            None,
        )
        if not geom_metadata:
            raise ValueError("Unable to parse geometry metadata from jsonld")

        description_metadata = [
            i for i in jsonld_response.get("@graph")
            if "dct:description" in i.keys()
        ][0]

        metadata = {
            "tiff_metadata": tiff_metadata,
            "geom_metadata": geom_metadata,
            "description_metadata": description_metadata
        }

        return metadata
    else:
        # only jsonld support.
        raise NotImplementedError()

# Item Creation 

An item is the second smallest STAC object we can create. If a collection is like a molecule than an item is an individual atom. Each item holds information about a single file, and includes a link to that file in the form of an asset. We're going to add lots of information to our item including extensions used to fill out data that doesnt fall under the native STAC spec. 

First we're going to use the metadata to get some basic properties. Let's take a look and see what we can learn from it:

In [37]:
# If the open.canada.ca site is down for maintenance use the Wayback Machine link below
#
metadata_url = 'https://web.archive.org/web/20200210170028/https://open.canada.ca/data/en/dataset/add1346b-f632-4eb9-a83d-a662b38655ad.jsonld'
# metadata_url = 'https://open.canada.ca/data/en/dataset/add1346b-f632-4eb9-a83d-a662b38655ad.jsonld'
metadata = get_metadata(metadata_url)
metadata


{'description_metadata': {'@id': 'https://web.archive.org/web/20200210170028/https://open.canada.ca/dataset/add1346b-f632-4eb9-a83d-a662b38655ad',
  'dcat:contactPoint': {'@id': '_:N9d213cb3f2af4cf18cf2eb3e7c0275d9'},
  'dcat:distribution': [{'@id': 'https://web.archive.org/web/20200210170028/https://open.canada.ca/dataset/add1346b-f632-4eb9-a83d-a662b38655ad/resource/4334d4fe-5314-43ab-b412-f164cd54af52'},
   {'@id': 'https://web.archive.org/web/20200210170028/https://open.canada.ca/dataset/add1346b-f632-4eb9-a83d-a662b38655ad/resource/3d3df246-3614-42c0-b754-c4ceb0c8587b'},
   {'@id': 'https://web.archive.org/web/20200210170028/https://open.canada.ca/dataset/add1346b-f632-4eb9-a83d-a662b38655ad/resource/1595fc60-b4ac-4ea3-a920-ff10b921fd5a'}],
  'dct:accrualPeriodicity': 'as_needed',
  'dct:description': 'This data publication contains a set of files in which areas affected by fire or by harvest from 1984 to 2015 are identified at the level of individual 30m pixels on the Landsat gri

The title, description, and date are all important parts of the item. Note that title and description are added into a dictionary called properties. The properties dict holds additional metadata that falls outside of the standard STAC item args. Check out the docs here for more info: https://pystac.readthedocs.io/en/1.0/api/pystac.html#pystac.Item

In [38]:
title = metadata["tiff_metadata"]["dct:title"]
description = metadata["description_metadata"]["dct:description"]
properties = {
    "title": title,
    "description": description,
}

start_datetime = datetime.strptime('1984', "%Y")  
end_datetime = datetime.strptime('2015', "%Y")

Here I use rasterio to extract and transform some important spatial data for our STAC. STAC spec requires data in epsg 4326, but our projection extension requires the native geometry and bbox, so we take both. 

In [39]:
#cog_href = 'CanLaD_20151984_latest_type_cog.tif' 
cog_href = 'https://ogc-madrid.s3.eu-central-1.amazonaws.com/CanLaD_20151984_latest_type_cog.tiff'

with rasterio.open(cog_href) as dataset:
  cog_bbox = list(dataset.bounds)
  cog_transform = list(dataset.transform)
  cog_shape = [dataset.height, dataset.width]

  transformer = Proj.from_crs(CRS.from_epsg(DATA_EPSG),
                              CRS.from_epsg(4326),
                              always_xy=True)
  bbox = list(
      transformer.transform_bounds(dataset.bounds.left,
                                    dataset.bounds.bottom,
                                    dataset.bounds.right,
                                    dataset.bounds.top))
  geometry = geojson_mapping(box(*bbox, ccw=True))

Now we'll declare and initialize our item, using the information we've collected so far. We pass in a unique id, our geometry and bounding box, start and end date times, the properties dictionary and an empy list of STAC extensions. This last list we'll then fill in using applicable extensions.

In [40]:
# Create item
item = pystac.Item(
  id='type_raster',
  geometry=geometry,
  bbox=bbox,
  datetime=start_datetime,
  properties=properties,
  stac_extensions=[],
)

item.common_metadata.start_datetime = start_datetime
item.common_metadata.end_datetime = end_datetime

You can run this cell to see the item spec in a sidebar, try it out!

In [41]:
item?


### Assets 

An asset is an object that contains a URI to the data associated with an item. In our case we will be adding two assets to each item in our collection. One asset is for the data itself while the other is for its metadata. Assets can either be created inside the `add_asset` function or created, then added after. You can see both methods here.  

In [42]:
item.add_asset(
  "metadata",
  pystac.Asset(
      href=metadata_url,
      media_type=pystac.MediaType.JSON,
      roles=["metadata"],
      title="Landsat Disturbance Metadata",
  ),
)
cog_asset = pystac.Asset(
  href= cog_href,
  media_type=pystac.MediaType.COG,
  roles=[
    "data",
    "labels",
    "labels-raster",
  ],
  title="Canada Landsat Disturbance COG",
)
item.add_asset('COG', cog_asset)


## Extensions

The STAC spec itself is fairly minimal, but due to its extensibility it can become much more comprehensive. STAC extensions allow us to add tons of different data that is associated with our dataset. Extensions are JSON fields that are associated with etiher items, assets, collection or catalogs. In most cases when you are creating a STAC you will need to dig into the extensions in order to properly describe your data. The overview page is found here: https://stac-extensions.github.io/.
One can add as many extensions as is applicable. 

For the purposes of this workshop we'll be focussing on item and asset extensions. 

### Projection Extension
We'll start with the projection extension which provides a way to describe Items whose assets are in a geospatial projection. More info about this extension can be found here: https://github.com/stac-extensions/projection

In [43]:
item_projection = ProjectionExtension.ext(item, add_if_missing=True)
item_projection?


In [44]:
item_projection.epsg = DATA_EPSG
item_projection.wkt2 = DATA_CRS_WKT
item_projection.bbox = cog_bbox
item_projection.transform = cog_transform
item_projection.shape = cog_shape

### Scientific Extension
The scientific extension is used when an item contains data linked to a scientific publication, it tells a user which publication it originates from and how to cite it. I grabbed the DOI and the citation from here: https://open.canada.ca/data/en/dataset/add1346b-f632-4eb9-a83d-a662b38655ad

You can find more info about the scientific extension here: https://github.com/stac-extensions/scientific

In [45]:
CITATION = 'Guindon, L.; Bernier, P.Y.; Gauthier, S.; Stinson, G.; Villemaire, P.; Beaudoin, A. 2018. Missing forest cover gains in boreal forests explained. Ecosphere, 9 (1) Article e02094.'
DOI = '10.1002/ecs2.2094'

item_sci = ScientificExtension.ext(item, add_if_missing=True)
item_sci.doi = DOI
item_sci.citation = CITATION

### Label Extension

Though intended to support labeled AOIs with ML models the label extension is useful for any data that contains labeled AOIs. We use it here to provide info on the the classes in our raster data. 
More info on the label extension can be found here: https://github.com/stac-extensions/label

In [46]:
CLASSIFICATION_VALUES = {1: 'fire',
                         2: 'harvest'}

item_label = LabelExtension.ext(item, add_if_missing=True)
item_label.label_type = LabelType.RASTER
item_label.label_tasks = [LabelTask.CLASSIFICATION]
item_label.label_properties = None
item_label.label_description = "Fire or Harvest"
item_label.label_classes = [LabelClasses.create(list(CLASSIFICATION_VALUES.values()), None)]


### File Info Extension 

Provides basic info such as size, data type and checksum for assets in an item or collection. The file extension can also be used to map our classification values. More info on the file extension can be found here: https://github.com/stac-extensions/file

In [53]:
from pystac.extensions.file import MappingObject

cog_asset_file = FileExtension.ext(cog_asset, add_if_missing=True)

mapping = [
  MappingObject.create(values=[value], summary=summary)
  for value, summary in CLASSIFICATION_VALUES.items()
]
cog_asset_file.values = mapping

with fsspec.open(cog_href) as file:
  size = file.size
  cog_asset_file.size = size

In [54]:
item.to_dict()

{'assets': {'COG': {'file:size': 299320339,
   'file:values': [{'summary': 'fire', 'values': [1]},
    {'summary': 'harvest', 'values': [2]}],
   'href': 'https://ogc-madrid.s3.eu-central-1.amazonaws.com/CanLaD_20151984_latest_type_cog.tiff',
   'roles': ['data', 'labels', 'labels-raster'],
   'title': 'Canada Landsat Disturbance COG',
   'type': <MediaType.COG: 'image/tiff; application=geotiff; profile=cloud-optimized'>},
  'metadata': {'href': 'https://web.archive.org/web/20200210170028/https://open.canada.ca/data/en/dataset/add1346b-f632-4eb9-a83d-a662b38655ad.jsonld',
   'roles': ['metadata'],
   'title': 'Landsat Disturbance Metadata',
   'type': <MediaType.JSON: 'application/json'>}},
 'bbox': [29.208147682613188,
  40.20704372347072,
  134.7641106438931,
  80.41031119985433],
 'geometry': {'coordinates': (((134.7641106438931, 40.20704372347072),
    (134.7641106438931, 80.41031119985433),
    (29.208147682613188, 80.41031119985433),
    (29.208147682613188, 40.20704372347072),
 

# Collection Creation

Next we're going to create a collection. A collection contains fields that describe a group of items that share properties and metadata. In our example our collection will hold the 'type' and 'latest year' raster data sets. 

In [55]:
title = metadata["tiff_metadata"]["dct:title"]
description = metadata["description_metadata"]["dct:description"]

license = "OGL-Canada-2.0"
license_link = Link(
  rel="license",
  target="https://open.canada.ca/en/open-government-licence-canada",
  title="Open Government Licence - Canada",
)

start_datetime = datetime.strptime('1984', "%Y")  
end_datetime = datetime.strptime('2015', "%Y")


CANLAD_PROVIDER = Provider(
    name="Open Government",
    roles=[
        ProviderRole.HOST,
        ProviderRole.LICENSOR,
        ProviderRole.PROCESSOR,
        ProviderRole.PRODUCER,
    ],
    url=
    "https://open.canada.ca/data/en/dataset/add1346b-f632-4eb9-a83d-a662b38655ad"
)


collection = pystac.Collection(
  id='CanLaD',
  title=title,
  description=description,
  providers=[CANLAD_PROVIDER],
  license=license,
  extent=pystac.Extent(
    pystac.SpatialExtent([bbox]),
    pystac.TemporalExtent([[start_datetime, end_datetime]]),
  ),
  catalog_type=pystac.CatalogType.RELATIVE_PUBLISHED,
)
collection.add_link(license_link)


## Collection Assets

Just like items, collection can also have assets. Here we add the metadata as an asset.

In [56]:
collection.add_asset(
  "metadata",
  pystac.Asset(
    href=metadata_url,
    media_type=pystac.MediaType.JSON,
    roles=["metadata"],
    title="Canada Landsat Disturbance metadata",
  )
)

## Collection Extensions

Collections can also have extensions, some of these are already familiar. Here we add the scientific, label and projection extensions just like we did before:

In [57]:
collection_label = LabelExtension.summaries(collection,
                                            add_if_missing=True)
collection_label.label_type = [LabelType.RASTER]
collection_label.label_tasks = [LabelTask.CLASSIFICATION]
collection_label.label_properties = None
collection_label.label_classes = [
    LabelClasses.create(list(CLASSIFICATION_VALUES.values()), None)
]

collection_proj = ProjectionExtension.summaries(collection,
                                                add_if_missing=True)
collection_proj.epsg = [DATA_EPSG]

collection_sci = ScientificExtension.ext(collection, add_if_missing=True)
collection_sci.doi = DOI
collection_sci.citation = CITATION

In [58]:
item.set_self_href('item.json')
collection.set_self_href('collection.json')
collection.add_item(item)


In [59]:
collection.to_dict()

{'assets': {'metadata': {'href': 'https://web.archive.org/web/20200210170028/https://open.canada.ca/data/en/dataset/add1346b-f632-4eb9-a83d-a662b38655ad.jsonld',
   'roles': ['metadata'],
   'title': 'Canada Landsat Disturbance metadata',
   'type': <MediaType.JSON: 'application/json'>}},
 'description': 'This data publication contains a set of files in which areas affected by fire or by harvest from 1984 to 2015 are identified at the level of individual 30m pixels on the Landsat grid. Details of the product development can be found in Guindon et al (2018).  \nThe change detection is based on reflectance-corrected yearly summer (July and August) Landsat mosaics from 1984 to 2015 created from individual scenes developed from USGS reflectance products  (Masek et al, 2006; Vermote et al, 2006). Briefly, the change detection method uses a six-year temporal signature centered on the disturbance year to identify fire, harvest and no change.  The signatures were derived from visually-interpre

In [60]:
item.to_dict()

{'assets': {'COG': {'file:size': 299320339,
   'file:values': [{'summary': 'fire', 'values': [1]},
    {'summary': 'harvest', 'values': [2]}],
   'href': 'https://ogc-madrid.s3.eu-central-1.amazonaws.com/CanLaD_20151984_latest_type_cog.tiff',
   'roles': ['data', 'labels', 'labels-raster'],
   'title': 'Canada Landsat Disturbance COG',
   'type': <MediaType.COG: 'image/tiff; application=geotiff; profile=cloud-optimized'>},
  'metadata': {'href': 'https://web.archive.org/web/20200210170028/https://open.canada.ca/data/en/dataset/add1346b-f632-4eb9-a83d-a662b38655ad.jsonld',
   'roles': ['metadata'],
   'title': 'Landsat Disturbance Metadata',
   'type': <MediaType.JSON: 'application/json'>}},
 'bbox': [29.208147682613188,
  40.20704372347072,
  134.7641106438931,
  80.41031119985433],
 'collection': 'CanLaD',
 'geometry': {'coordinates': (((134.7641106438931, 40.20704372347072),
    (134.7641106438931, 80.41031119985433),
    (29.208147682613188, 80.41031119985433),
    (29.2081476826131

In [62]:
item.validate()
collection.validate()

item.save_object()
collection.save_object()

# Next Steps

Now it's time for you to take control and give it a shot yourself. Using the code above and the second raster dataset you can create another item and add it to our collection. There's not too much you'll need to change: the `cog_href` and `CLASSIFICATION_VALUES`. 

In [ ]:
NEW_CLASSIFICATION_VALUES = {1985 : '1985',
                             1986 : '1986',
                             1987 : '1987',
                             1988 : '1988',
                             1989 : '1989',
                             1990 : '1990',
                             1991 : '1991',
                             1992 : '1992',
                             1993 : '1993',
                             1994 : '1994',
                             1995 : '1995',
                             1996 : '1996',
                             1997 : '1997',
                             1998 : '1998',
                             1999 : '1999',
                             2000 : '2000',
                             2001 : '2001',
                             2002 : '2002',
                             2003 : '2003',
                             2004 : '2004',
                             2005 : '2005',
                             2006 : '2006',
                             2007 : '2007',
                             2008 : '2008',
                             2009 : '2009',
                             2010 : '2010',
                             2011 : '2011',
                             2012 : '2012',
                             2013 : '2013',
                             2014 : '2014',
                             2015 : '2015'}